In [1]:
!pip install selenium
!pip install chromedriver-py==94.0.4606.41
!pip install requests

# libraries
import os
import pandas as pd
import shutil
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from bs4 import BeautifulSoup as bs
from chromedriver_py import binary_path
import requests
import warnings; warnings.filterwarnings("ignore")

# unhide all rows and columns
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 1000)

In [2]:
years = list(range(2013,2024))
url_start = "https://www.basketball-reference.com/leagues/NBA_{}.html"

In [3]:
dfs = []

for year in years:
    url = url_start.format(year)
    data = requests.get(url)
    
    with open("season/{}.html".format(year), "w+") as f:
        f.write(data.text)

In [4]:
for year in years:
    with open("season/{}.html".format(year)) as f:
        page = f.read()
        
    soup = bs(page, "html.parser")
    soup.find("tr", class_ = "over_header").decompose()
    team_stats_table = soup.find_all(id = "per_game-team")
    team_stat = pd.read_html(str(team_stats_table))[0]
    team_stat["Year"] = year
    
    dfs.append(team_stat)
    team_stats = pd.concat(dfs)

In [6]:
team_stats = team_stats[team_stats["Team"] != 'League Average']

In [8]:
team_stats['Team'] = team_stats['Team'].str.replace('*', '')

In [10]:
team_stats = team_stats.drop(team_stats.columns[0], axis=1)


In [26]:
team_advanced_stats_dfs = []

for year in years:
    with open("season/{}.html".format(year)) as f:
        page = f.read()
        
    soup = bs(page, "html.parser")
    soup.find("tr", class_ = "over_header").decompose()
    team_advanced_stats_table = soup.find_all(id = "advanced-team")
    team_advanced_stat = pd.read_html(str(team_advanced_stats_table))[0]
    team_advanced_stat["Year"] = year
    
    team_advanced_stats_dfs.append(team_advanced_stat)
    team_advanced_stats = pd.concat(team_advanced_stats_dfs)

In [28]:
team_advanced_stats = team_advanced_stats.drop(team_advanced_stats.columns[0], axis=1)

In [30]:
team_advanced_stats['Team'] = team_advanced_stats['Team'].str.replace('*', '')

In [32]:
team_advanced_stats = team_advanced_stats[team_advanced_stats["Team"] != 'League Average']

In [ ]:
team_advanced_stats = team_advanced_stats.drop('Unnamed: 17', axis=1)
team_advanced_stats = team_advanced_stats.drop('Unnamed: 22', axis=1)
team_advanced_stats = team_advanced_stats.drop('Unnamed: 27', axis=1)

In [39]:
team_advanced_stats = team_advanced_stats.drop('Arena', axis=1)

In [42]:
team_advanced_stats.to_csv('team_advanced_stats.csv')

In [43]:
url_playoff_start = 'https://www.basketball-reference.com/playoffs/NBA_{}.html'

po_per_game_dfs = []

for year in years:
    url = url_playoff_start.format(year)
    data = requests.get(url)
    
    with open("playoff/{}.html".format(year), "w+") as f:
        f.write(data.text)

In [44]:
for year in years:
    with open("playoff/{}.html".format(year)) as f:
        page = f.read()
        
    soup = bs(page, "html.parser")
    soup.find("tr", class_ = "over_header").decompose()
    team_po_stats_table = soup.find_all(id = "per_game-team")
    team_po_stat = pd.read_html(str(team_po_stats_table))[0]
    team_po_stat["Year"] = year
    
    po_per_game_dfs.append(team_po_stat)
    team_po_stats = pd.concat(po_per_game_dfs)

In [46]:
team_po_stats = team_po_stats.dropna(subset=['MP'])

In [49]:
team_po_stats = team_po_stats[team_po_stats["Tm"] != 'League Average']

In [50]:
team_po_stats = team_po_stats.rename(columns={'Tm': 'Team'})

In [53]:
team_po_stats.to_csv('team_po_stats.csv')

In [54]:
po_adv_stat_dfs = []

In [55]:
for year in years:
    with open("playoff/{}.html".format(year)) as f:
        page = f.read()
        
    soup = bs(page, "html.parser")
    soup.find("tr", class_ = "over_header").decompose()
    po_adv_stats_table = soup.find_all(id = "advanced-team")
    po_adv_stat = pd.read_html(str(po_adv_stats_table))[0]
    po_adv_stat["Year"] = year
    
    po_adv_stat_dfs.append(po_adv_stat)
    po_adv_stats = pd.concat(po_adv_stat_dfs)

In [57]:
po_adv_stats = po_adv_stats.drop(po_adv_stats.columns[0], axis=1)
po_adv_stats = po_adv_stats[po_adv_stats["Tm"] != 'League Average']
po_adv_stats = po_adv_stats.drop('Unnamed: 15', axis=1)
po_adv_stats = po_adv_stats.drop('Unnamed: 20', axis=1)

In [58]:
po_adv_stats = po_adv_stats.dropna(subset=['Age'])

In [60]:
po_adv_stats = po_adv_stats.rename(columns={'Tm': 'Team'})

In [62]:
po_adv_stats.to_csv('po_adv_stats.csv')